In [ ]:
import os

import numpy as np
import pandas as pd

import lightning as L
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader, random_split

from model import BaselineModel
from dataset import EpilepsyDataset
from dataset_formatter import DatasetFormatter

In [ ]:
epilepsy_dataset = EpilepsyDataset(
    '/workspace/labels.csv', 
    '/workspace/new_data/', 
    ['Acc x', 'Acc y', 'Acc z', 'Acc Mag', 'EDA', 'BVP'],
    23040,
)

In [ ]:
train_size = int(0.7 * len(epilepsy_dataset))
test_size = len(epilepsy_dataset) - train_size

train_dataset, test_dataset = random_split(epilepsy_dataset, [train_size, test_size])

train_dataloader = DataLoader(epilepsy_dataset, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
model = BaselineModel(7)

In [ ]:
wandb_logger = WandbLogger()

trainer = L.Trainer(max_epochs=13, logger=wandb_logger)
trainer.fit(model=model, train_dataloaders=train_dataloader)

In [ ]:
trainer.test(dataloaders=test_dataloader, ckpt_path='best')